# Loading Libraries  

In [11]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt
import os.path
import getpass
import platform
from pandasql import sqldf

In [12]:
username = getpass.getuser()

print("Username: ", username)
pathPart1 = "/Users/"
pathPart2 = "/Github/SCGProject/Datasets/"
complePath = pathPart1+username+pathPart2

if platform.system() == "Darwin":
    if(username == "marcovinciguerra"):
        dfCambio = pd.read_csv(
            complePath+"/CorrectedDatasets/dfCambio.csv")

        dfClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dfClienti.csv")

        dfConsumi = pd.read_csv(
             complePath + "/CorrectedDatasets/dfConsumi.csv")

        dfCostoOrarioConsuntivo = pd.read_csv(
            complePath+"/CorrectedDatasets/dfConsuntivo.csv")

        dfCostoOrarioBudget = pd.read_csv(
            complePath+"CorrectedDatasets/dfCostoOrario.csv")

        dfImpiegoRisorse = pd.read_csv(
         complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

        dftuttiClienti = pd.read_csv(
         complePath+"/CorrectedDatasets/dftuttiClienti.csv")

        dfVendite = pd.read_csv(
         complePath+"/CorrectedDatasets/dfVendite.csv")

    elif(username == "davidguzman"):
            pathPart1 = "/Users/"
            pathPart2 = "/documents/Github/SCGProject/Datasets/"
            complePath = pathPart1+username+pathPart2

            dfCambio = pd.read_csv(
                complePath+"/CorrectedDatasets/dfCambio.csv")

            dfClienti = pd.read_csv(
                complePath+"/CorrectedDatasets/dfClienti.csv")

            dfConsumi = pd.read_csv(
                complePath + "/CorrectedDatasets/dfConsumi.csv")

            dfCostoOrarioConsuntivo = pd.read_csv(
                complePath+"/CorrectedDatasets/dfConsuntivo.csv")

            dfCostoOrarioBudget = pd.read_csv(
                complePath+"CorrectedDatasets/dfCostoOrario.csv")

            dfImpiegoRisorse = pd.read_csv(
            complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

            dftuttiClienti = pd.read_csv(
            complePath+"/CorrectedDatasets/dftuttiClienti.csv")

            dfVendite = pd.read_csv(
            complePath+"/CorrectedDatasets/dfVendite.csv")

elif platform.system() == "Linux":
    complePath = "/home/alinux/unibg/git_repo/SCGProject"
    pathPart1 = "/Users/"
    pathPart2 = "/documents/Github/SCGProject/Datasets/"
    complePath = pathPart1+username+pathPart2
    complePath = '/home/alinux/unibg/git_repo/SCGProject/Datasets/'

    dfCambio = pd.read_csv(
        complePath+"/CorrectedDatasets/dfCambio.csv")

    dfClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dfClienti.csv")

    dfConsumi = pd.read_csv(
        complePath + "/CorrectedDatasets/dfConsumi.csv")

    dfCostoOrarioConsuntivo = pd.read_csv(
        complePath+"/CorrectedDatasets/dfConsuntivo.csv")

    dfCostoOrarioBudget = pd.read_csv(
        complePath+"CorrectedDatasets/dfCostoOrario.csv")

    dfImpiegoRisorse = pd.read_csv(
        complePath+"/CorrectedDatasets/dfImpiegoRisorse.csv")

    dftuttiClienti = pd.read_csv(
        complePath+"/CorrectedDatasets/dftuttiClienti.csv")

    dfVendite = pd.read_csv(
        complePath+"/CorrectedDatasets/dfVendite.csv")


Username:  alinux


# Analisi qtà articoli a BUDGET

In [59]:
dfConsumi.rename(columns = {'Budget/cons':'status'}, inplace = True)
qtaArtBudget = sqldf('select NrArticolo, count(*) as qta from dfConsumi where status = "BUDGET" group by NrArticolo')

dfVendite.rename(columns = {'budget/cons':'status'}, inplace = True)
qtaArtVendBudget = sqldf('select NrArticolo, count(*) as qta from dfVendite where status = "BUDGET" group by NrArticolo')

artProdeVendBudget = sqldf('select P.NrArticolo from qtaArtBudget P join qtaArtVendBudget V on P.NrArticolo = V.NrArticolo')

# ho trovato articoli che hanno consumato risorse a budget ma non sono stati messi tra le vendite a budget
artProdeNONVendBudget = sqldf('select NrArticolo from qtaArtBudget where NrArticolo not in (select NrArticolo from artProdeVendBudget)')

consumiB = sqldf('select * from dfConsumi where status = "BUDGET"')


# prendo gli articoli che usano come materia prima almeno un articolo prodotto internamente
# sqldf('select NrMovimento, status, C.NrArticolo, CodiceMP, NrDocumento, QuantityMPImpiegata, ImportoCostoTOTALE from consumiB C join artProdeNONVendBudget A on C.CodiceMP = A.NrArticolo')

costoPerArticoloBudget = sqldf('select NrArticolo, sum(ImportoCostoTOTALE) as costoMpPerArt from consumiB where NrArticolo not in (select NrArticolo from artProdeNONVendBudget) group by NrArticolo')
sqldf('select sum(costoMpPerArt) as costoTotBudget from costoPerArticoloBudget')

,costoTotBudget
0,172660.4


# Includo il prezzo processi

In [63]:
status = "BUDGET"
impRisBudget = sqldf('select * from dfImpiegoRisorse where budgetConsuntivo = "BUDGET"')

impRisBudgetConPzuOrar = sqldf('select * from impRisBudget I join dfCostoOrarioBudget C on I.Risorsa = C.Risorsa and I.NrAreaProduzione = C.AreaProduzione')
impCostoProcBudget = sqldf('select NrArticolo, AreaProduzione, Risorsa, (TempoRisorsa*CostoOrario) as costoProc from impRisBudgetCOnPzuOrar')

impCostoProcPerArticoloBudget = sqldf('select NrArticolo, sum(costoProc) as costoProc from impCostoProcBudget group by NrArticolo')
#faccio il totale dei processi per articolo e il totale di mp per articolo da consumi e gli metto insieme

costoTotPerArticoloBudget = sqldf('select A.NrArticolo, costoProc+costoMpPerArt as costoTot from impCostoProcPerArticoloBudget A join costoPerArticoloBudget B on A.NrArticolo=B.NrArticolo')
#print(dfCostoOrarioBudget)

# Cerco di calcolare il prezzo di vendita a budget

In [69]:
venditeB = sqldf('select * from dfVendite where status = "BUDGET"')

cambioB = sqldf('select * from dfCambio where Anno = "BUDGET"')
valutaCliente = sqldf('select Nr, TassoCambioMedio from dfClienti C join cambioB CB on C.Valuta = CB.CodiceValuta ')

clienteConValuta = sqldf('select * from venditeB B join valutaCliente VC on B.NrOrigine = Nr')

venditeBinEuro = sqldf('select NrMovimento, status, NrArticolo, NrOrigine, Quantity, (ImportoVenditaValutaLocaleTOTALEVENDITA/TassoCambioMedio) as PzuEuro from clienteConValuta')

pzuPerArticoloBudget = sqldf('select NrArticolo, sum(PzuEuro) as pzuEuroPerArticolo from venditeBinEuro group by NrArticolo')
#sqldf('select sum(pzuEuroPerArticolo) as ricaviBudget from pzuPerArticoloBudget')


totdiff = sqldf('select A.NrArticolo, (pzuEuroPerArticolo-costoTot) as diff from costoTotPerArticoloBudget A join pzuPerArticoloBudget B on A.NrArticolo=B.NrArticolo')

sqldf('select sum(diff) from totdiff') #sembra ragionevole
#todo sotrarre dal pzuEuroPerArticolo il costo per la lavorazione interna di articoli che sono stati usati come MP per altri prodotti

,sum(diff)
0,122985.816003


# Confronto brutale inutile perché non ho considerato le lavorazioni

In [70]:
costoArtBudget = sqldf('select NrArticolo, sum(ImportoCostoTOTALE) as costoArt from consumiB group by NrArticolo')

sqldf('select C.NrArticolo, (V.PzuEuro-C.costoArt*V.Quantity) as diff from costoArtBudget C join venditeBinEuro V on C.NrArticolo = V.NrArticolo')

sqldf('select * from costoArtBudget C join venditeBinEuro V on C.NrArticolo = V.NrArticolo')


,NrArticolo,costoArt,NrMovimento,status,NrArticolo,NrOrigine,Quantity,PzuEuro
0,ART0000018,5.15,28836,BUDGET,ART0000018,C00003,2,90.000000
1,ART0000041,92.00,28839,BUDGET,ART0000041,C00003,4,380.000000
2,ART0000042,104.00,28840,BUDGET,ART0000042,C00003,4,600.000000
3,ART0000043,36.06,28841,BUDGET,ART0000043,C00003,2,230.000000
4,ART0000115,434.44,28499,BUDGET,ART0000115,C00026,2,610.000000
...,...,...,...,...,...,...,...,...
1204,ART0005136,136.54,33619,BUDGET,ART0005136,C00015,4,0.243112
1205,ART0005136,136.54,34119,BUDGET,ART0005136,C00004,6,4.376013
1206,ART0005136,136.54,34516,BUDGET,ART0005136,C00004,10,0.972447
1207,ART0005136,136.54,34890,BUDGET,ART0005136,C00158,64,2.593193


In [71]:
# articoli da capire dove sono finiti, sono state consumate risorse per farli ma non sono stati utilizzati per la produzione di un'altro articolo oppure venduti

#verifico che siano effettivamente loro
#x = sqldf('select distinct CodiceMP from prodInternaPerArticoliBudget group by CodiceMP')
#articoliDaCapireDoveSonoFiniti = sqldf('select NrArticolo from artProdeNONVendBudget where NrArticolo not in (select CodiceMP from x)')

#sqldf('select * from consumiB C1 join prodInternaPerArticoliBudget C2 on C1.CodiceMP = C2.NrArticolo')

